In [1]:
from pymatgen.ext.matproj import MPRester
import numpy as np
import pandas as pd
import scipy.constants
import math

C:\Users\Amelia\miniconda3\lib\site-packages\pymatgen\analysis\phase_diagram.py:24: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


Import The Materials Porject API key

In [2]:
with open('key.txt') as f:
    key = f.readlines()
    key = key[0]

In [3]:
mpr = MPRester(key)

C:\Users\Amelia\miniconda3\lib\site-packages\pymatgen\ext\matproj.py:179: UserWarning: You are using the legacy MPRester, which is deprecated. If you are not a power user, ie., you do not have a lot of legacy code that uses many different MPRester methods, it is recommended you get a new API key from the new Materials Project front end. Once you use get your new API key, using the new API key will automatically route you to using the new MPRester.
  warnings.warn(


Query data from The Materials Project with species contain Li and phonon data

In [4]:
data = mpr.query(
    criteria = {"has": "phonons","elements": {"$all": ["Li"]}},
    properties = ["task_id", "pretty_formula"],
)

In [5]:
data = pd.DataFrame(data)

Extract a Li key for filtering Li pdos from an example species

In [6]:
LiYS2_dos=mpr.get_phonon_dos_by_material_id('mp-15788')
Li_dos_LiYS2 = LiYS2_dos.get_element_dos()
Li_key = [*Li_dos_LiYS2.keys()][0]

In [7]:
def get_fav (task_id):
    '''Get a temperature independent phonon band centre value'''
    t_dos_obj = mpr.get_phonon_dos_by_material_id ( task_id )
    el_dos_obj = t_dos_obj.get_element_dos()
    Li_dos_obj = el_dos_obj [Li_key]
    Li_f = Li_dos_obj.frequencies 
    Li_d = Li_dos_obj.densities
    fav_elements = Li_f * Li_d
    fav = np.sum(fav_elements)
    norm = np.sum(Li_d)
    fav = fav / norm
    return fav

In [8]:
k = scipy.constants.k
hbar = scipy.constants.hbar

def get_favT (task_id, Tmin, Tmax, Tstep):
    '''Get a list of temperature dependent phonon band centre values or a single value at a given T'''
    favT = []
    
    if ((Tmin != None) and (Tmax == None) and (Tstep == None)):
        Temperature = [Tmin] 
    elif ((Tmin != None) and (Tmax != None) and (Tstep != None)):
        Temperature = [*range(Tmin, Tmax, Tstep)]
    else:
        print("Tmin is expected as a value, Tmax and Tstep could be None")
        Temperature = None
        
    t_dos_obj = mpr.get_phonon_dos_by_material_id ( task_id )
    el_dos_obj = t_dos_obj.get_element_dos()
    Li_dos_obj = el_dos_obj [Li_key]
    Li_f = Li_dos_obj.frequencies 
    Li_d = Li_dos_obj.densities   
        
    for T in Temperature:
        ph_numbers = []
        beta = 1/(k*T)
        for frequency in Li_f:
            if frequency < 0:
                ph_numbers.append(1)
                
            else:
                energy = frequency * hbar * 10 ** 12
                denominator = math.exp(energy*beta) - 1
                ph_number = 1/denominator
                ph_numbers.append(ph_number)
                
        
        fav_elements = Li_f * Li_d * ph_numbers
        norm_elements = Li_d * ph_numbers
        
        fav = np.sum(fav_elements)
        norm = np.sum(norm_elements)
        fav = fav / norm
        favT.append(fav)
        
    return favT

Create a temp dataframe for calling materials id

In [9]:
temp = data.copy()

Create an empty column for temperature independent phonon band centre

In [10]:
data['fav'] = np.nan

Calculate all temperature independent phonon band centres and write it in the dataframe, data is the dataframe query from The Materials Project with materials id in it

In [11]:
index = [*range (0, data.index[-1]+1)]
for i in index:
    fav = get_fav(temp['task_id'][i])
    data['fav'][i] = fav

C:\Users\Amelia\AppData\Local\Temp\ipykernel_4780\3421046949.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['fav'][i] = fav


Calculate temperature dependent phonon band centres and write it in the dataframe, examples for single room temperature value and a selected temperature range

In [12]:
Tmin = 298
Tmax = None
Tstep = None

if ((Tmin != None) and (Tmax == None) and (Tstep == None)):
    Temperature = [Tmin] 
elif ((Tmin != None) and (Tmax != None) and (Tstep != None)):
    Temperature = [*range(Tmin, Tmax, Tstep)]
else:
    print("Tmin is expected as a value, Tmax and Tstep could be None")
    Temperature = None

    
index = [*range (0, data.index[-1]+1)]
T_val = pd.DataFrame(columns = Temperature)
for i in index:
    favT = get_favT(temp['task_id'][i], Tmin, Tmax, Tstep)
    T_val.loc[i] = favT
    
data = pd.concat([data, T_val], axis = 1)

In [13]:
Tmin = 400
Tmax = 1000
Tstep = 100

if ((Tmin != None) and (Tmax == None) and (Tstep == None)):
    Temperature = [Tmin] 
elif ((Tmin != None) and (Tmax != None) and (Tstep != None)):
    Temperature = [*range(Tmin, Tmax, Tstep)]
else:
    print("Tmin is expected as a value, Tmax and Tstep could be None")
    Temperature = None

    
index = [*range (0, data.index[-1]+1)]
T_val = pd.DataFrame(columns = Temperature)
for i in index:
    favT = get_favT(temp['task_id'][i], Tmin, Tmax, Tstep)
    T_val.loc[i] = favT
    
data = pd.concat([data, T_val], axis = 1)


First frequency that has non-zero Li DOS

(i) ignore imaginary

In [14]:
def first_f_i (task_id):
    '''get first frequency that has non-zero Li DOS and non-imaginary '''
    t_dos_obj = mpr.get_phonon_dos_by_material_id ( task_id )
    el_dos_obj = t_dos_obj.get_element_dos()
    Li_dos_obj = el_dos_obj [Li_key]
    Li_f = Li_dos_obj.frequencies 
    Li_d = Li_dos_obj.densities
    df = pd.DataFrame({'Li_f': Li_f, 'Li_d': Li_d})
    df = df[(df['Li_d'] > 0)&(df['Li_f']> 0)]
    df = df.reset_index()
    f = df.loc[0,'Li_f']
    return f

(ii) take the absolute value 

In [15]:
def first_f_a (task_id):
    '''get first frequency (absolute) that has non-zero Li DOS'''
    t_dos_obj = mpr.get_phonon_dos_by_material_id ( task_id )
    el_dos_obj = t_dos_obj.get_element_dos()
    Li_dos_obj = el_dos_obj [Li_key]
    Li_f = Li_dos_obj.frequencies 
    Li_d = Li_dos_obj.densities
    Li_f = abs(Li_f)
    key = np.nonzero(Li_d)[0][0]
    f = Li_f[key]
    return f

Create an empty column

In [16]:
data['first_f_a'] = np.nan

Example for getting the data

In [17]:
index = [*range (0, data.index[-1]+1)]
for i in index:
    f = first_f_a(temp['task_id'][i])
    data['first_f_a'][i] = f

C:\Users\Amelia\AppData\Local\Temp\ipykernel_4780\3049303763.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['first_f_a'][i] = f


 Li DOS width relative to the total width

In [18]:
def relative_w (task_id):
    '''get Li DOS width relative to the total width'''
    
    t_dos_obj = mpr.get_phonon_dos_by_material_id ( task_id )
    el_dos_obj = t_dos_obj.get_element_dos()
    Li_dos_obj = el_dos_obj [Li_key]
    total_f = t_dos_obj.frequencies
    total_d = t_dos_obj.densities
    Li_f = Li_dos_obj.frequencies 
    Li_d = Li_dos_obj.densities
    
    df_1 = pd.DataFrame({'total_f': total_f, 'total_d': total_d})
    f_1 = df_1.loc[0,'total_f']
    f_2 = df_1.iloc[-1].loc['total_f']
    total_f_d = f_2 - f_1
    
    df_2 = pd.DataFrame({'Li_f': Li_f, 'Li_d': Li_d}) 
    f_1 = df_2.loc[0,'Li_f']
    f_2 = df_2.iloc[-1].loc['Li_f']
    Li_f_d = f_2 - f_1
    
    relative_w = Li_f_d / total_f_d
    
    return relative_w

Create an empty column

In [19]:
data['relative_w'] = np.nan

Example for getting the data

In [20]:
index = [*range (0, data.index[-1]+1)]
for i in index:
    w = relative_w(temp['task_id'][i])
    data['relative_w'][i] = w

C:\Users\Amelia\AppData\Local\Temp\ipykernel_4780\1125960793.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['relative_w'][i] = w


In [21]:
data

,task_id,pretty_formula,fav,298,400,500,600,700,800,900,first_f_a,relative_w
0,mp-15788,LiYS2,6.202698,5.830757,5.837647,5.841635,5.844279,5.846159,5.847566,5.848657,0.072377,1.0
1,mp-996962,LiAgO2,10.015853,8.664702,8.701859,8.723256,8.737392,8.747425,8.754915,8.760719,0.072377,1.0
2,mp-23818,BaLiH3,10.604209,9.834129,9.857356,9.870780,9.879667,9.885985,9.890706,9.894367,0.072377,1.0
3,mp-5840,LiScO2,9.787954,7.522111,7.576124,7.607432,7.628196,7.642973,7.654025,7.662603,0.072377,1.0
4,mp-33526,LiBiS2,5.534439,5.222082,5.227304,5.230326,5.232329,5.233754,5.234819,5.235646,0.072377,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
135,mp-961698,LiZnP,9.613426,7.592647,7.642841,7.671675,7.690696,7.704182,7.714243,7.722036,0.072377,1.0
136,mp-3731,LiNbO3,9.629222,8.785166,8.808411,8.821815,8.830677,8.836970,8.841670,8.845314,0.072377,1.0
137,mp-7611,Li2CaGeO4,13.180677,11.515282,11.576673,11.611680,11.634665,11.650912,11.663004,11.672353,0.072377,1.0
138,mp-8405,Li3LaSb2,8.907796,7.856691,7.883062,7.898248,7.908280,7.915401,7.920717,7.924837,0.072377,1.0


Write the data into csv file

In [22]:
data.to_csv('data_small.csv')